In [1]:
from astropy.modeling import models, Fittable1DModel, Parameter
from astropy.modeling import fitting
import astropy.units as u
import astropy.constants as const

from spectacle.spectrum import SpectrumModel
from spectacle.registries import line_registry

import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook
# plt.rcParams["figure.figsize"] = [8, 16]
# plt.rcParams["figure.dpi"] = 80

## Control Spectrum

This spectrum acts as the standard to which the dynamic spectrum will be compared in the correlation metrics.

In [3]:
import astropy.units as u

wavelength = np.linspace(1210, 1250, 1001) * u.Angstrom
velocity = np.linspace(-800, 800, 201) * u.Unit("km/s")

In [4]:
from spectacle.models import TauProfile
from astropy.modeling.models import Voigt1D
from astropy.modeling.fitting import LevMarLSQFitter

print(line_registry['name'])
print(line_registry[line_registry['name'] == 'HI1216'])
line = line_registry.with_name('HI1215')
print(line['wave'])

spectrum = SpectrumModel(center=line['wave']).add_line(name='HI1216', 
                                                       column_density=1e14 * u.Unit('1/cm2'),
                                                       v_doppler=1e7 * u.Unit('cm/s'),
                                                       delta_v=1000 * u.Unit('km/s'))

voigt = Voigt1D(x_0=0.0001, amplitude_L=0.17, fwhm_G=40, fwhm_L=8)

fitter = LevMarLSQFitter()

new_voigt = fitter(voigt, velocity, spectrum.flux(velocity))

f, ax = plt.subplots()

ax.plot(velocity, spectrum.flux(velocity))
ax.scatter(velocity, voigt(velocity.value))

print(new_voigt)
ax.plot(velocity, new_voigt(velocity))



# fwhm = line.fwhm(velocity, velocity=True)
# print(fwhm)

# ax.axvline(line['lambda_0']-fwhm*0.5)
# ax.axvline(line['lambda_0']+fwhm*0.5)

# ax.axvline(line['lambda_0']-fwhm)
# ax.axvline(line['lambda_0']+fwhm)

# mask = spectrum.line_mask(velocity)
# ax.scatter(velocity[mask], spectrum.tau.velocity_space(velocity, center=line['lambda_0'])[mask])
vmn, vmx = spectrum._line_model.dv90()

# for line in spectrum._line_model:
#     print(line.dv90(wavelength))

  name  
--------
  HI1216
  HI1026
   HI973
   HI950
   HI938
   HI931
   HI926
   HI923
   HI921
   HI919
     ...
NiII1467
NiII1455
NiII1393
NiII1370
NiII1317
CuII1368
CuII1359
ZnII2063
ZnII2026
GeII1602
GaII1414
Length = 329 rows
 name     wave   osc_str    gamma   
        Angstrom                    
------ --------- ------- -----------
HI1216 1215.6701  0.4164 626500000.0
Correct name is HI1216 given HI1215.
1215.6701
Correct name is HI1216 given HI1216.


<IPython.core.display.Javascript object>

Model: Voigt1D
Inputs: ('x',)
Outputs: ('y',)
Model set size: 1
Parameters:
         x_0       amplitude_L      fwhm_L        fwhm_G   
        km / s                      km / s        km / s   
    ------------- ------------- ------------- -------------
    -170.00242017 9.49250865098 2445.89377471 31916.5231377


In [5]:
ax.axvline(vmn.value)
ax.axvline(vmx.value)

In [6]:
fwhm = spectrum._line_model.fwhm().value
ax.axvline(-fwhm*0.5, color='r')
ax.axvline(fwhm*0.5, color='r')

In [7]:
from spectacle.models import LineFinder

# nspec = SpectrumModel()
# nspec = nspec.from_data(wavelength, spectrum.tau(wavelength), smooth=False)
nspec = LineFinder()#line_list=["HI1216"])
print(nspec)
print(nspec(velocity, spectrum.tau(velocity)))
print(nspec)
fitter = LevMarLSQFitter()
fit_nspec = fitter(nspec, velocity, spectrum.tau(velocity))

print(fit_nspec)

f, ax = plt.subplots()

# ax.plot(wavelength, spectrum.flux(wavelength))
ax.plot(wavelength, nspec(wavelength))

Model: LineFinder
Inputs: ('x', 'y')
Outputs: ('y',)
Model set size: 1
Parameters:
    threshold min_distance width  center 
                                 Angstrom
    --------- ------------ ----- --------
          0.5         30.0  10.0      0.0
[-800. -792. -784. -776. -768. -760. -752. -744. -736. -728. -720. -712.
 -704. -696. -688. -680. -672. -664. -656. -648. -640. -632. -624. -616.
 -608. -600. -592. -584. -576. -568. -560. -552. -544. -536. -528. -520.
 -512. -504. -496. -488. -480. -472. -464. -456. -448. -440. -432. -424.
 -416. -408. -400. -392. -384. -376. -368. -360. -352. -344. -336. -328.
 -320. -312. -304. -296. -288. -280. -272. -264. -256. -248. -240. -232.
 -224. -216. -208. -200. -192. -184. -176. -168. -160. -152. -144. -136.
 -128. -120. -112. -104.  -96.  -88.  -80.  -72.  -64.  -56.  -48.  -40.
  -32.  -24.  -16.   -8.    0.    8.   16.   24.   32.   40.   48.   56.
   64.   72.   80.   88.   96.  104.  112.  120.  128.  136.  144.  152.
  160.  168.  176. 

AttributeError: 'NoneType' object has no attribute 'outputs'